# LangGraph 100本ノック

このJupyter Notebookは、LangGraphの基礎から応用までを体系的に学ぶための100問の演習問題とその解答、解説を提供します。LangGraphを用いた複雑なエージェントワークフローの構築スキルを習得しましょう。

## 目次

*   **第1章: グラフの基本要素 (問題 001-020):** `StateGraph`, `State`, ノード, エッジの定義。最もシンプルなグラフの構築と実行。
*   **第2章: グラフの制御フロー (問題 021-040):** 条件付きエッジによる分岐、自己修正ループ、エラーハンドリング、人間による介入(`Interrupt`)。
*   **第3章: ツールを使うシングルエージェント (問題 041-060):** `ToolNode`の活用、ReAct型エージェント、計画と実行(Plan-and-Execute)型エージェントの構築。
*   **第4章: マルチエージェント・ワークフロー (問題 061-090):** スーパーバイザー型によるタスク割り振り、複数エージェントによる対話シミュレーション、階層型エージェント。
*   **第5章: 発展的なグラフ技術 (問題 091-100):** Agent Swarm、永続化(`Checkpointer`)による状態保存と再開、ストリーミング、カスタムエージェント。

In [ ]:
# === ライブラリのインストール ===
# ご利用になるLLMプロバイダーに応じて、以下のコメントを解除して実行してください。
# !%pip install -qU langchain langchain_openai langchain_google_vertexai langchain_google_genai langchain_anthropic langchain_aws boto3

# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

# === APIキー/環境変数の設定 ===
import os
try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION
    # AZURE_OPENAI_DEPLOYMENT_NAME はクライアント初期化時に直接渡すため、ここでは環境変数設定は必須ではない

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")
    # GOOGLE_APPLICATION_CREDENTIALS は通常環境変数で設定されるか、Colabでは自動認証

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

    # os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID # これは通常不要
    # os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION # これも通常不要

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: # Colabの場合、boto3が認証情報をうまく見つけられない場合があるので明示的な設定を推奨
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    # BedrockはIAMロールなど他の認証方法もあるため、キーがなくても動作する可能性はある。
    # ただし、ここでは明示的な設定がない場合は警告またはエラーを出すことを検討できる。
    # 今回は、キーがない場合でも初期化は試みるが、リージョンは必須とする。
    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_REGION_NAME"] = AWS_REGION_NAME


# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME, # 上で取得・設定済み
        openai_api_version=OPENAI_API_VERSION, # 上で取得・設定済み
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    llm = ChatVertexAI(model_name="gemini-1.5-flash-001", temperature=0, project=PROJECT_ID, location=LOCATION)
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    # 利用可能なモデル名の例: "gemini-1.5-flash-latest", "gemini-1.5-pro-latest", "gemini-pro"
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0, convert_system_message_to_human=True)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock
    llm = BedrockChat(
        model_id="anthropic.claude-3-haiku-20240307-v1:0", # Bedrockで利用するモデルID
        region_name=AWS_REGION_NAME, # 上で取得・設定済み
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    model_attr = getattr(llm, 'model', getattr(llm, 'model_name', getattr(llm, 'model_id', None)))
    if model_attr:
        print(f"LLM Model: {model_attr}")


### ■ 問題001: 最小構成のLangGraphグラフの構築

LangGraphの最も基本的な構成要素である`StateGraph`と`State`を理解し、シンプルなグラフを構築してみましょう。この問題では、入力された文字列をそのまま出力するだけの、単一のノードを持つグラフを作成します。

In [ ]:
# 解答欄001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    print(f"simple_node: {state["messages"][-1].content}")
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("simple_node", simple_node)

# エントリポイントの設定
workflow.set_entry_point("simple_node")

# 終了ポイントの設定
workflow.add_edge("simple_node", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}
for s in app.stream(inputs):
    print(s)

# 最終結果の確認
final_state = app.invoke(inputs)
print(f"Final State: {final_state}")


<details><summary>解答001</summary>

``````python
# 解答001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    # グラフの状態を保持する辞書
    # ここでは、入力メッセージを保持する
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    # 入力されたメッセージをそのまま返すノード
    print(f"simple_node: {state["messages"][-1].content}")
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("simple_node", simple_node)

# エントリポイントの設定
workflow.set_entry_point("simple_node")

# 終了ポイントの設定
workflow.add_edge("simple_node", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}
for s in app.stream(inputs):
    print(s)

# 最終結果の確認
final_state = app.invoke(inputs)
print(f"Final State: {final_state}")
``````
</details>

<details><summary>解説001</summary>

#### この問題のポイント
*   **学習内容:** この問題では、`StateGraph`、`TypedDict`を用いた`State`の定義、`add_node`、`set_entry_point`、`add_edge`、`END`といったLangGraphの最も基本的なAPIを学びます。また、`Annotated`と`add_messages`を使ってメッセージ履歴を管理する方法も理解します。
*   **コード解説:**
    *   `GraphState`は、グラフ全体で共有される状態を定義します。`TypedDict`を使うことで、状態のスキーマを明確にできます。`messages: Annotated[list, add_messages]`は、LangChainのメッセージ形式のリストを状態として持ち、新しいメッセージが追加されるたびに自動的にリストの末尾に追加されるように設定しています。
    *   `simple_node`関数は、グラフのノードとして機能します。`state`引数として現在のグラフの状態を受け取り、新しい状態を辞書として返します。ここでは、入力された最後のメッセージをそのまま返しています。
    *   `StateGraph(GraphState)`でグラフのインスタンスを作成し、`GraphState`で定義した状態スキーマを渡します。
    *   `workflow.add_node("simple_node", simple_node)`で、`simple_node`関数を`simple_node`という名前のノードとしてグラフに追加します。
    *   `workflow.set_entry_point("simple_node")`は、グラフの実行が開始される最初のノードを指定します。
    *   `workflow.add_edge("simple_node", END)`は、`simple_node`の実行が完了したらグラフを終了することを示します。`END`はLangGraphが提供する特別な終了ノードです。
    *   `app = workflow.compile()`で、定義したワークフローを実行可能なアプリケーションにコンパイルします。
    *   `app.stream(inputs)`は、グラフの実行過程をストリーミングで受け取ることができます。`app.invoke(inputs)`は、グラフの実行が完了した最終状態を返します。
---
</details>

### ■ 問題002: 複数のノードを持つシーケンシャルグラフの構築

前の問題で学んだ基本的なグラフ構築に加えて、複数のノードを直列に接続し、データがノード間をどのように流れるかを理解しましょう。ここでは、入力された文字列を加工する2つのノード（例：大文字化、逆順化）を持つグラフを作成します。

In [ ]:
# 解答欄002

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    print(f"uppercase_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    print(f"reverse_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("uppercase", uppercase_node)
workflow.add_node("reverse", reverse_node)

# エントリポイントの設定
workflow.set_entry_point("uppercase")

# エッジの追加 (直列接続)
workflow.add_edge("uppercase", "reverse")
workflow.add_edge("reverse", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [HumanMessage(content="Hello LangGraph")]}
for s in app.stream(inputs):
    print(s)

final_state = app.invoke(inputs)
print(f"Final State: {final_state}")


<details><summary>解答002</summary>

``````python
# 解答002

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    # 最新のメッセージを大文字に変換するノード
    last_message_content = state["messages"][-1].content
    print(f"uppercase_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    # 最新のメッセージを逆順にするノード
    last_message_content = state["messages"][-1].content
    print(f"reverse_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("uppercase", uppercase_node)
workflow.add_node("reverse", reverse_node)

# エントリポイントの設定
workflow.set_entry_point("uppercase")

# エッジの追加 (直列接続)
workflow.add_edge("uppercase", "reverse")
workflow.add_edge("reverse", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [HumanMessage(content="Hello LangGraph")]}
for s in app.stream(inputs):
    print(s)

final_state = app.invoke(inputs)
print(f"Final State: {final_state}")
``````
</details>

<details><summary>解説002</summary>

#### この問題のポイント
*   **学習内容:** 複数のノードを`add_edge`で直列に接続する方法と、ノード間で状態がどのように引き継がれるかを学びます。`HumanMessage`と`AIMessage`を使って、メッセージの送信元を明示する方法も理解します。
*   **コード解説:**
    *   `uppercase_node`と`reverse_node`は、それぞれ入力メッセージを大文字化、逆順化する処理を行います。重要なのは、各ノードが新しい`AIMessage`を作成して状態に返す点です。これにより、次のノードは前のノードの処理結果を`state["messages"][-1]`で取得できます。
    *   `workflow.add_edge("uppercase", "reverse")`は、`uppercase`ノードの実行が完了したら、次に`reverse`ノードを実行するように指示します。このようにして、処理の流れを定義します。
    *   入力メッセージを`HumanMessage`として渡すことで、ユーザーからの入力であることを明示しています。ノードからの出力は`AIMessage`として返され、メッセージ履歴にAIの応答として記録されます。
---
</details>

### ■ 問題003: 条件付きエッジによる分岐の導入

LangGraphの強力な機能の一つである条件付きエッジを導入し、グラフの実行パスを動的に制御する方法を学びましょう。ここでは、入力された数値が偶数か奇数かによって、異なる処理を行うグラフを作成します。

In [ ]:
# 解答欄003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int # 新たに数値を保持する状態を追加

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    # 入力メッセージから数値を抽出し、状態に保存するノード
    try:
        num = int(state["messages"][-1].content)
        print(f"check_number: Extracted number {num}")
        return {"number": num}
    except ValueError:
        print("check_number: Invalid input, not a number.")
        return {"number": 0} # エラー時は0として扱うか、適切なエラーハンドリングを実装

def even_node(state: GraphState):
    # 偶数だった場合の処理ノード
    print(f"even_node: Number {state["number"]} is even.")
    return {"messages": [AIMessage(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    # 奇数だった場合の処理ノード
    print(f"odd_node: Number {state["number"]} is odd.")
    return {"messages": [AIMessage(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    # 数値の状態に基づいて次のノードを決定する
    if state["number"] % 2 == 0:
        print("Routing to even_node")
        return "even_node"
    else:
        print("Routing to odd_node")
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("check_number", check_number)
workflow.add_node("even_node", even_node)
workflow.add_node("odd_node", odd_node)

# エントリポイントの設定
workflow.set_entry_point("check_number")

# 条件付きエッジの追加
workflow.add_conditional_edges(
    "check_number", # 遷移元のノード
    route_number,   # ルーター関数
    {
        "even_node": "even_node", # ルーター関数の戻り値とノード名のマッピング
        "odd_node": "odd_node"
    }
)

# 各分岐からの終了エッジ
workflow.add_edge("even_node", END)
workflow.add_edge("odd_node", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 偶数のテスト ---")
inputs_even = {"messages": [HumanMessage(content="42")]}
for s in app.stream(inputs_even):
    print(s)
final_state_even = app.invoke(inputs_even)
print(f"Final State (Even): {final_state_even}")

print("\n--- 奇数のテスト ---")
inputs_odd = {"messages": [HumanMessage(content="77")]}
for s in app.stream(inputs_odd):
    print(s)
final_state_odd = app.invoke(inputs_odd)
print(f"Final State (Odd): {final_state_odd}")
``````
</details>

<details><summary>解説003</summary>

#### この問題のポイント
*   **学習内容:** `add_conditional_edges`を使用して、グラフの実行パスを動的に制御する方法を学びます。ルーター関数がどのように次のノードを決定するのか、そして状態が分岐間でどのように共有されるかを理解します。
*   **コード解説:**
    *   `GraphState`に`number`という新しいキーを追加し、入力された数値を保持するようにしました。
    *   `check_number`ノードは、入力メッセージから数値を抽出し、`number`状態を更新します。
    *   `even_node`と`odd_node`は、それぞれ偶数と奇数だった場合の最終処理を行います。
    *   `route_number`関数がルーターとして機能します。この関数は現在の`state`を受け取り、次に実行すべきノードの名前（文字列）を返します。LangGraphは、この戻り値に基づいて適切なエッジを辿ります。
    *   `workflow.add_conditional_edges("check_number", route_number, {"even_node": "even_node", "odd_node": "odd_node"})`は、`check_number`ノードの後に`route_number`関数を実行し、その戻り値が`"even_node"`なら`even_node`へ、`"odd_node"`なら`odd_node`へ遷移するように設定しています。
---
</details>

### ■ 問題004: グラフ内でのLLMの利用（シンプルなチャットボット）

LangGraphのノード内で大規模言語モデル（LLM）を呼び出す方法を学び、シンプルなチャットボットを構築しましょう。ここでは、ユーザーからの入力に対してLLMが応答を生成し、その応答を返すグラフを作成します。

In [ ]:
# 解答欄004
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
import os # osはAPIキー設定のコメントアウト部分で使われているので残しても良いが、直接は不要になる

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def llm_node(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    # ノートブック冒頭で初期化された共通の `llm` 変数を使用します。
    print(f"llm_node: Calling LLM with messages: {state['messages']}")
    response = llm.invoke(state["messages"]) # 共通llmを使用
    print(f"llm_node: LLM response: {response.content}")
    return {"messages": [response]} # responseはAIMessageオブジェクトを期待

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("llm_responder", llm_node)

# エントリポイントの設定
workflow.set_entry_point("llm_responder")

# 終了ポイントの設定
workflow.add_edge("llm_responder", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
# 最初のメッセージはHumanMessageであると想定
inputs = {"messages": [HumanMessage(content="こんにちは、あなたの名前は何ですか？")]}
for s in app.stream(inputs):
    print(s)

final_state = app.invoke(inputs)
print(f"Final State: {final_state}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [HumanMessage(content="今日の天気は？")]}
for s in app.stream(inputs2):
    print(s)

final_state2 = app.invoke(inputs2)
print(f"Final State: {final_state2}")


<details><summary>解答004</summary>

``````python
# 解答004

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
import os # osはAPIキー設定のコメントアウト部分で使われているので残しても良いが、直接は不要になる

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def llm_node(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    # ノートブック冒頭で初期化された共通の `llm` 変数を使用します。
    print(f"llm_node: Calling LLM with messages: {state['messages']}")
    response = llm.invoke(state["messages"]) # 共通llmを使用
    print(f"llm_node: LLM response: {response.content}")
    return {"messages": [response]} # responseはAIMessageオブジェクトを期待

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("llm_responder", llm_node)

# エントリポイントの設定
workflow.set_entry_point("llm_responder")

# 終了ポイントの設定
workflow.add_edge("llm_responder", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
# 最初のメッセージはHumanMessageであると想定
inputs = {"messages": [HumanMessage(content="こんにちは、あなたの名前は何ですか？")]}
for s in app.stream(inputs):
    print(s)

final_state = app.invoke(inputs)
print(f"Final State: {final_state}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [HumanMessage(content="今日の天気は？")]}
for s in app.stream(inputs2):
    print(s)

final_state2 = app.invoke(inputs2)
print(f"Final State: {final_state2}")
``````
</details>

<details><summary>解説004</summary>

このノートブックでは、様々なLLMプラットフォーム（OpenAI, Azure OpenAI, Google Cloud Vertex AI, Google Gemini (Gemini API), Anthropic Claude, Amazon Bedrockなど）を簡単に切り替えて試せるように設計されています。
ノートブックの冒頭にある `LLM_PROVIDER` 変数で使用したいLLMを選択し、対応するAPIキーや環境変数を設定するだけで、この問題を含む全てのLLM呼び出し箇所で選択したLLMが利用されます。
選択した `LLM_PROVIDER` に応じて、必要なAPIキーが設定されているか（環境変数またはGoogle Colabのシークレット経由）、ノートブック起動時にチェックされます。

ここでは、ノートブックの先頭で設定・初期化された共通の `llm` 変数を使用して、LLMに質問をしています。
`llm.invoke()` という統一されたインターフェースで、どのLLMプロバイダーを利用しているかに関わらず、同じようにLLMを呼び出すことができます。
これにより、特定のLLMサービスに依存しない、より汎用的なコードを作成するメリットを手軽に体験できます。

もしエラーが発生した場合は、ノートブック冒頭の `LLM_PROVIDER` の設定、および選択したプロバイダーに応じたAPIキーや環境変数の設定（例: `OPENAI_API_KEY`, `GOOGLE_API_KEY`, `AZURE_OPENAI_ENDPOINT`など）が正しく行われているかを確認してください。
各プロバイダー固有の設定項目（例えばVertex AIのProject ID、AzureのDeployment Name、Bedrockのリージョンなど）も見直してください。
プロバイダーによっては、`pip install` コマンドで対応するライブラリ (例: `langchain-google-genai`) がインストールされているかも確認点です。

</details>


### ■ 問題005: グラフの可視化とデバッグ

構築したLangGraphグラフの構造を視覚的に確認し、デバッグに役立てる方法を学びましょう。ここでは、これまでに作成したグラフのいずれか（例：問題003の条件分岐グラフ）を可視化し、その構造を理解します。

In [ ]:
# 解答欄005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    try:
        num = int(state["messages"][-1].content)
        return {"number": num}
    except ValueError:
        return {"number": 0}

def even_node(state: GraphState):
    return {"messages": [AIMessage(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    return {"messages": [AIMessage(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    if state["number"] % 2 == 0:
        return "even_node"
    else:
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

workflow.add_node("check_number", check_number)
workflow.add_node("even_node", even_node)
workflow.add_node("odd_node", odd_node)

workflow.set_entry_point("check_number")

workflow.add_conditional_edges(
    "check_number",
    route_number,
    {
        "even_node": "even_node",
        "odd_node": "odd_node"
    }
)

workflow.add_edge("even_node", END)
workflow.add_edge("odd_node", END)

app = workflow.compile()

# --- グラフの可視化 ---
# グラフを画像として表示
# graphvizがインストールされている必要があります: pip install pygraphviz pydotplus graphviz
# また、システムにGraphvizがインストールされている必要があります。
# Windows: https://graphviz.org/download/
# Mac: brew install graphviz
# Linux: sudo apt-get install graphviz
try:
    display(Image(app.get_graph().draw_png()))
    print("グラフが正常に可視化されました。")
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの実行と結果表示 (オプション) ---
# 可視化したグラフが正しく動作するか確認するために、再度実行してみる
print("\n--- 偶数のテスト (可視化後の確認) ---")
inputs_even = {"messages": [HumanMessage(content="10")]}
for s in app.stream(inputs_even):
    print(s)
final_state_even = app.invoke(inputs_even)
print(f"Final State: {final_state_even}")
``````
</details>

<details><summary>解説005</summary>

#### この問題のポイント
*   **学習内容:** `app.get_graph().draw_png()`を使用してLangGraphのグラフ構造を画像として可視化する方法を学びます。これにより、複雑なグラフのデバッグや理解が容易になります。
*   **コード解説:**
    *   この問題では、問題003で作成した条件分岐グラフを再利用しています。これは、可視化の有用性を示すのに適した例だからです。
    *   `app.get_graph()`は、コンパイルされたグラフの内部表現を取得します。
    *   `.draw_png()`メソッドは、そのグラフ構造をPNG画像としてバイト列で返します。この機能を利用するには、システムにGraphvizがインストールされている必要があります。また、Pythonの`pygraphviz`や`pydotplus`といったライブラリも必要になる場合があります。
    *   `IPython.display.Image`と`display`を使うことで、Jupyter Notebook内で直接画像をレンダリングして表示できます。
    *   `try-except`ブロックでGraphvizのインストール状況によるエラーをハンドリングし、ユーザーに適切なメッセージを表示するようにしています。グラフが複雑になるにつれて、この可視化機能はデバッグや設計の確認に不可欠となります。
---
</details>